In [9]:
import torch.nn as nn
import torch.nn.functional as F
import torch

fc = nn.Linear(in_features=1*28*28, out_features=10) # 全连接层，参数 bias 代表是否加上偏执，默认为 True

print(fc)
print(fc.parameters)

x_input = torch.randn(1, 28, 28, 1)
# [B, C, H, W]

x_input = x_input.view(1, -1)
print(x_input.shape)

x_output = fc(x_input)
print(x_output.shape)

Linear(in_features=784, out_features=10, bias=True)
<bound method Module.parameters of Linear(in_features=784, out_features=10, bias=True)>
torch.Size([1, 784])
torch.Size([1, 10])


In [10]:
class test:
    def __call__(self, x):
        print("forward test", x)
        return 

a = test()
print(a(5))

forward test 5
None


In [11]:
class test:
    def hello(self, x):
        print("forward test", x)
        return 
    
a = test()
print(a(5))

TypeError: 'test' object is not callable

可以发现，其实实例直接传参是不合法的操作，但是 Python 类的内置 \_\_call\_\_ 函数的作用就是接受这样的参数，当传递的时候会执行这个类内的函数，而这个函数指向的就是 forward，这就是为什么直接用实例传到参数会执行类内的 forward 函数，所以构建的类要对 forward 函数进行重写

forward 在基类里面只是一个占位符，实现的继承类里面不写 forward 的话会抛出异常，也就是子类需要是实现他，而不是重写进行覆盖，和 Dataset 类延申的子类一样，Dataset 类内的 \_\_getitem\_\_ 函数也是没有实现的，但是也是必须要写的，在继承类里面需要实现

In [ ]:
out = F.relu(x_output)

print(out.shape)
print(out)

out = F.sigmoid(x_output)

print(out.shape)
print(out)

torch.Size([1, 10])
tensor([[0.3609, 0.2339, 0.0406, 0.0000, 0.0000, 0.3531, 0.3402, 0.0000, 0.0814,
         0.0000]], grad_fn=<ReluBackward0>)
torch.Size([1, 10])
tensor([[0.5892, 0.5582, 0.5101, 0.2705, 0.4244, 0.5874, 0.5842, 0.3721, 0.5203,
         0.4942]], grad_fn=<SigmoidBackward0>)


torch.nn.functional 简称 F 是 Pytorch 内部的函数式的接口模块，提供了大量的构建神经网络时常用的计算函数，比如线性变换、卷积、池化、归一化、激活函数、损失函数等，这些函数不具备可学习参数，主要用于直接调用计算逻辑，而不是作为独立的层对象

nn.Module 封装了参数（nn.Parameter）和前向计算的逻辑，torch.nn.functional 里面的函数本身 不保存参数，允许手动传入参数，常见的 F 包里面的函数除去上面举例的 relu 和 sigmoid 还有下面的一些

In [ ]:
out = F.leaky_relu(x_output, negative_slope=0.1) # 允许部分的负值漏出去的 relu，全部学习的话，直接看官方文档，或者用到再进行学习

print(out.shape)
print(out) 

out = F.tanh(x_output)

print(out.shape)
print(out)

print(x_output)
softmax = F.softmax(x_output, dim=-1)

print(softmax.shape)
print(softmax)

a = torch.tensor([[1, 2, 3], [4, 5, 6], [7, 8, 9]], dtype=float)

softmax = F.softmax(a, dim=0)
print(softmax)

torch.Size([1, 10])
tensor([[ 0.3609,  0.2339,  0.0406, -0.0992, -0.0305,  0.3531,  0.3402, -0.0523,
          0.0814, -0.0023]], grad_fn=<LeakyReluBackward0>)
torch.Size([1, 10])
tensor([[ 0.3460,  0.2298,  0.0406, -0.7582, -0.2956,  0.3391,  0.3277, -0.4802,
          0.0812, -0.0232]], grad_fn=<TanhBackward0>)
tensor([[ 0.3609,  0.2339,  0.0406, -0.9919, -0.3047,  0.3531,  0.3402, -0.5233,
          0.0814, -0.0232]], grad_fn=<AddmmBackward0>)
torch.Size([1, 10])
tensor([[0.1389, 0.1223, 0.1008, 0.0359, 0.0714, 0.1378, 0.1360, 0.0574, 0.1050,
         0.0946]], grad_fn=<SoftmaxBackward0>)
tensor([[0.0024, 0.0024, 0.0024],
        [0.0473, 0.0473, 0.0473],
        [0.9503, 0.9503, 0.9503]], dtype=torch.float64)


In [ ]:
labels = torch.tensor([1])
print(x_output.shape, labels.shape)
score = F.cross_entropy(x_output, labels)
print(score)

torch.Size([1, 10]) torch.Size([1])
tensor(2.4866, grad_fn=<NllLossBackward0>)


交叉熵对矩阵的输入有严格的限制要求

多分类：[B, C]，[B]

In [ ]:
x = torch.tensor([[1, 2, 3, 4], [5, 6, 7, 8], [6, 7, 8, 9], [3, 4, 5, 6]], dtype=torch.float32)
x = x.reshape((1, 1, 4, -1)) # [B, C, H, W]
w = torch.randn(2, 1, 3, 3) # [OC, IC, H, W]
print(x.shape)
print(w)
out = F.conv2d(x, weight=w, bias=None, stride=1, padding=0)
print(out)

print(out.sum(), out.mean(), out.std())
print(F.normalize(out, dim=3)) # 模长变为 1，dim=-1，第一个维度是 Batch
out = (out - out.mean()) / out.std() # 均值为 0，方差为 1
print(out)

# 需要注意的是，多多的理解一下高位的矩阵的维度计算，三维还是能理解的，但是更高的维度就需要拓展了

torch.Size([1, 1, 4, 4])
tensor([[[[ 1.3765,  0.7075,  0.4137],
          [ 1.0668,  0.8018,  1.7712],
          [ 0.8032,  1.0854,  0.6792]]],


        [[[-0.3063,  1.0093, -1.9599],
          [-0.5453,  0.1735,  0.5203],
          [ 0.8356,  0.9866,  1.8117]]]])
tensor([[[[44.4269, 53.1323],
          [50.3538, 59.0592]],

         [[24.2025, 26.7279],
          [ 8.4214, 10.9469]]]])
tensor(277.2709) tensor(34.6589) tensor(19.6443)
tensor([[[[0.6415, 0.7672],
          [0.6488, 0.7610]],

         [[0.6712, 0.7413],
          [0.6097, 0.7926]]]])
tensor([[[[ 0.4972,  0.9404],
          [ 0.7990,  1.2421]],

         [[-0.5323, -0.4037],
          [-1.3356, -1.2071]]]])


写到这里再次复述一下上面的重点问题，nn.Xxx 和 nn.functional.xxx 的实际功能是相同的，比图 nn.Cvn2d 和 nn.functional.cvn2d 都是进行卷积，运行效率也是相似的

很显然完全一样的话是不合理的，因为没必要两个库函数是完全一样的功能，主要的不同点的，nn.functional.xxx 是函数接口，nn.Xxx 是 nn.functional.xxx 的类封装，并且 nn.Xxx 都继承自同一个祖先，nn.Moudle

nn.Xxx 除了具有 nn.functional.xxx 的功能之外，内部还附有，nn.Moudle 相关的属性和方，eg. train()，eval()

形式上看，nn.functional.xxx 的函数是小写字母开头，nn.Xxx 是函数是大写字母开头

nn.functional.xxx 是 API 函数接口而 nn.Xxx 是对原先的 API 的类封装

所有的 nn.Xxx 都继承于共同的祖先 nn.Moudle，这一点导致了 nn.Xxx 除了具有 nn.functional.xxx 的功能之外，内部附带了 nn.Moudle 的相关属性和方法，例如上面提到的 train()，eval()

nn.Xxx 继承于 nn.Moudle 能很好的和 nn.Sequential 结合使用，但是 nn.functional.xxx 无法和 nn.Sequential 结合使用

nn.Xxx 需要先实例化并传入参数，然后以函数调用的方式调入实例化的对象并传入输入的数据，nn.functional.xxx 同时传入输入数据和 weight、bias 等其他的参数

nn.Xxx 不需要自己定义和管理 weight，但是 nn.functional.xxx 需要自己定义 weight，每次调用的时候需要手动传入，不利于代码的复用

当执行，nn.Conv2d 的时候，会先调用里面的初始化函数，初始化函数的第一行调用了 nn.Moudle 的初始化函数，nn.Moudle 初始化函数里面初始化 _parameters，_moudles 等容器，然后接着执行 Conv2d 的参数注册

In [ ]:
# 伪代码
class Conv2d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, ...):
        super().__init__()
        self.weight = nn.Parameter(...)
        self.bias = nn.Parameter(...)

class Module:
    def __init__(self):
        self._parameters = OrderedDict()
        self._buffers = OrderedDict()
        self._modules = OrderedDict()
        self.training = True

上面是伪代码，继承 Module 的初始化是为了初始化框架，自己的初始化是为了保存参数，相比 nn.Xxx 函数 nn.functional.xxx 函数的参数需要自己来写非常的麻烦

In [ ]:
# 创建 model 是时候，nn 会自动的创建相应的参数 parameter，并自动的累加到模型的 Parameter 成员列表里面

# 反向传播需要被更新的称之为 Parameters，不需要被更新的称之为 buffers

# in_features 的数量，决定参数的个数，加上偏置，输入的维度为 in_features + 1，out_features 是数量，决定了全连接层中的神经元的个数，每个神经元只有一个输出

# W 的总参数量是 in_features * out_features，b 的总参数量的 1 * out_features

下面介绍手动创建参数的办法，模型才会有注册列表，不注册的话不能参与梯度下降的优化，nn.Xxx 函数会自动的注册当前的参数到注册表里面，不用显式的定义出类下面的 w 矩阵

In [29]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Linear(nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.w = nn.Parameter(torch.randn(out_features, in_features))
        self.b = nn.Parameter(torch.randn(out_features))

    def forward(self, x):
        return F.linear(x, self.w, self.b)

model = Linear(3, 1)

# 查看注册的参数列表
for name, param in model.named_parameters():
    print(name, param.shape, param.requires_grad)


x_input = torch.tensor([[1., 1., 1.]])
x_input = x_input.reshape(3, 1)

w = torch.randn((3, 1))

print(w.requires_grad)  # False
w = nn.Parameter(w)
print(w.requires_grad)  # True
b = nn.Parameter(torch.randn(3))

print(x_input.shape, w.shape)

fc = F.linear(x_input, weight=w, bias=b)

print(fc)

w torch.Size([1, 3]) True
b torch.Size([1]) True
False
True
torch.Size([3, 1]) torch.Size([3, 1])
tensor([[-1.3750, -2.4481, -1.3936],
        [-1.3750, -2.4481, -1.3936],
        [-1.3750, -2.4481, -1.3936]], grad_fn=<AddmmBackward0>)


nn.Module 是 pyTorch 所有组件的共同祖先，他既可以是一层 nn.Conv2d 也可以是很多层组成的模型，完全取决于自定义的 mynet，同意发基类，所有的层/模型都继承自他，比如 

Conv2d、Linear、BantchNorm、Sequential 容器和机制也是由他负责，管理与训练相关的对象，比如 Parameter（可训练参数）、buffer（非可训练但是要随着模型保存的张量）

递归管理子模块，设备迁移，精度变换，训练和评估模式的转换等，还有前向调用协议 \_\_call\_\_ 调用到 forward 等

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class MyBlock(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.conv = nn.Conv2d(in_ch, out_ch, 3, padding=0) # [output_channels, input_channels, kernels, ...]
        # 注册一个 buffer（非训练参数，但要随模型保存/迁移）
        self.register_buffer("coord_bias", torch.zeros(1, out_ch, 1, 1)) # 从后往前（右对齐）比维度，只要每一对对应维度相等，或其中一个为 1，就能广播

    def forward(self, x):
        out = self.conv(x) + self.coord_bias  # buffer 参与前向，但不被优化
        print(out.shape)
        return out

class MyNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.block1 = MyBlock(3, 16)
        self.head = nn.Linear(16*8*8, 10)

    def forward(self, x):
        x = self.block1(x)         # 模块嵌套
        x = F.adaptive_avg_pool2d(x, (8,8))
        x = x.flatten(1)
        return self.head(x)        # 调用线性层

net = MyNet()

print(net)

ipt = torch.randn(16, 3, 224, 224)

print(net(ipt).shape)

# 1) 参数与 buffer
print("=== parameters ===")
for n, p in net.named_parameters():
    print(n, p.shape, p.requires_grad)

print("\n=== buffers ===")
for n, b in net.named_buffers():
    print(n, b.shape)

# 2) 模块层级
print("\n=== modules ===")
for m in net.modules():
    print(type(m).__name__)

# 3) 状态字典（保存/加载用）
sd = net.state_dict()

for k, v in sd.items():
    print(k, v.shape)

# 4) 设备/精度迁移
net = net.to("cuda", dtype=torch.float32)  # 参数/buffer 全部到 GPU

# for x in sd 假设 sd 是一个字典的话，只会迭代字典的键，等价于 for x in sd.keys()，得带键值需要用 items()


<bound method Module.parameters of MyNet(
  (block1): MyBlock(
    (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1))
  )
  (head): Linear(in_features=1024, out_features=10, bias=True)
)>
MyNet(
  (block1): MyBlock(
    (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1))
  )
  (head): Linear(in_features=1024, out_features=10, bias=True)
)
torch.Size([16, 16, 222, 222])
torch.Size([16, 10])
=== parameters ===
block1.conv.weight torch.Size([16, 3, 3, 3]) True
block1.conv.bias torch.Size([16]) True
head.weight torch.Size([10, 1024]) True
head.bias torch.Size([10]) True

=== buffers ===
block1.coord_bias torch.Size([1, 16, 1, 1])

=== modules ===
MyNet
MyBlock
Conv2d
Linear
block1.coord_bias torch.Size([1, 16, 1, 1])
block1.conv.weight torch.Size([16, 3, 3, 3])
block1.conv.bias torch.Size([16])
head.weight torch.Size([10, 1024])
head.bias torch.Size([10])


In [ ]:
def parameters(self, recurse=True):
    for _, param in self.named_parameters(recurse=recurse):
        yield param
# model.parameters 只返回参数值不返回名字，model.named_parameters 返回名字和对象二者的关系可以大致看作上面函数展示的样子

# 前者常用于传递个优化器的优化 optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

# 后者常用于打印观察参数列表

定义的输入维度在前输出维度在后输出的参数列表就相反了的原因是，定义的时候写 (in, out) 更自然，如果反过来定义（nn.Linear(out, in)），虽然形状对，但语义不自然：

在神经网络设计中我们总是思考“输入几维 → 输出几维”，所以构造函数写 (in_features, out_features) 更符合人类直觉。PyTorch 内部再自动转成 [out, in] 的存储方式，所以输出参数矩阵的时候就是反的了

因为计算的时候需要转置，存储就是输出在前

下面介绍 nn.Sequential 构建神经网络的一般步骤，上面讲解了 nn.Module 还有常见的层的组件，现在把他们串在一起，很想 transforms 里面的 Compose。nn.Sequential 是一个有序的容器

该类将按照传入构造器的顺序，依次创建相应的函数，并记录在 Sequential 类对象的数据结构中，同时以神经网络模块为元素的有序字典也可以作为传入参数

因此，Sequential可以看成是有多个函数运算对象，串联成的神经网络，其返回的是Module类型的神经网络对象

In [ ]:
ipt = torch.randn(16, 28*28)

model = nn.Sequential(
    nn.Linear(28*28, 32),
    nn.ReLU(),
    nn.Linear(32, 10),
    nn.Softmax(dim=1)
)
print(type(model))
print(model)
print(model(ipt))

<class 'torch.nn.modules.container.Sequential'>
Sequential(
  (0): Linear(in_features=784, out_features=32, bias=True)
  (1): ReLU()
  (2): Linear(in_features=32, out_features=10, bias=True)
  (3): Softmax(dim=1)
)
tensor([[0.1154, 0.0831, 0.0818, 0.1071, 0.0676, 0.0993, 0.0987, 0.1452, 0.1026,
         0.0992],
        [0.1145, 0.0816, 0.0809, 0.1087, 0.0620, 0.1180, 0.1386, 0.1089, 0.0952,
         0.0917],
        [0.1157, 0.1139, 0.0764, 0.0783, 0.0857, 0.0809, 0.1332, 0.1124, 0.1007,
         0.1030],
        [0.0887, 0.0756, 0.0827, 0.0984, 0.0668, 0.1601, 0.1548, 0.1122, 0.0950,
         0.0657],
        [0.0934, 0.1006, 0.0960, 0.0915, 0.1100, 0.1031, 0.1257, 0.1185, 0.0815,
         0.0796],
        [0.0893, 0.0921, 0.1103, 0.1189, 0.0647, 0.1425, 0.1255, 0.0834, 0.0723,
         0.1011],
        [0.1124, 0.0889, 0.0811, 0.1003, 0.0856, 0.1086, 0.1134, 0.0974, 0.0948,
         0.1174],
        [0.0946, 0.1101, 0.0751, 0.1137, 0.0857, 0.1167, 0.0996, 0.1470, 0.0812,
         0.

In [ ]:
# def forward(self, x):
#     for module in self._modules.values():
#         x = module(x)
#     return x
# nn.Sequential 内部做的事情比较简单，上面就是，但是其只是简单的串联适用于简单的线性堆叠网络

适合用 Sequential 的情况：网络结构简单；不需要多输入、多分支；每一层依次执行；典型场景：多层感知机（MLP）、普通卷积堆叠

不适合 Sequential 的情况：网络有跳跃连接（skip connection）；有多输入或多输出；有条件执行逻辑（如 if/else 或循环）；需要自定义中间变量或计算统计量

In [ ]:
# 也可以以字典的形式，串联函数运算，构建串行执行的神经网络
from collections import OrderedDict
import torch.nn as nn

model = nn.Sequential(OrderedDict([
    ('h1', nn.Linear(28*28, 32)),
    ('relu1', nn.ReLU()),
    ('h2', nn.Linear(32, 10)),
    ('softmax', nn.Softmax(dim=1))
]))

print(type(model))
print(model)

<class 'torch.nn.modules.container.Sequential'>
Sequential(
  (h1): Linear(in_features=784, out_features=32, bias=True)
  (relu1): ReLU()
  (h2): Linear(in_features=32, out_features=10, bias=True)
  (softmax): Softmax(dim=1)
)


最后讲解的就是自定义神经网络了，也就是继承 nn.Moudle 类，继承的原因的，需要注册到参数列表还有，初始化参数列表，可使用相关的方法和函数等

函数都要加上 self 是原因是，当执行 model = TestNet()，创建实例之后，执行 model(23, 32)，其实执行的是 TestNet(model, 23, 32)，也就是将实例对象作为第一个参数传进去了，不创建实例的话

直接使用 TestNet(10, 32)，初始化和其他的函数就不用加上 self 了，但是 pyTorch 要求一定要实例化的 model 才能调用其相关的函数，不实例化每次 .parameters 都是新的参数，没办法训练

\_\_init\_\_ 创建实例的时候就执行了，\_\_call\_\_ 引导的 forward 需要 model(x) 的时候才会执行

In [76]:
class TestNet(nn.Module):
    def __init__(self, input_channels, output_channels):
        super().__init__()
        self.hidden_channels = input_channels * 4
        self.linear1 = nn.Linear(input_channels, self.hidden_channels)
        self.relu1 = nn.ReLU()
        self.linear2 = nn.Linear(self.hidden_channels, output_channels)
        self.softmax = nn.Softmax(dim=1)
    def forward(self, x):
        print(x.size())
        x = x.view(x.size()[0], -1)
        x = self.linear1(x)
        x = self.relu1(x)
        x = self.linear2(x)
        x = self.softmax(x)
        return x

model = TestNet(28*28, 32)
print(model.parameters)
x_input = torch.randn(2, 1, 28, 28)
x_output = model(x_input)
print(x_output.shape)

<bound method Module.parameters of TestNet(
  (linear1): Linear(in_features=784, out_features=3136, bias=True)
  (relu1): ReLU()
  (linear2): Linear(in_features=3136, out_features=32, bias=True)
  (softmax): Softmax(dim=1)
)>
torch.Size([2, 1, 28, 28])
torch.Size([2, 32])


剩余的一些函数

torch.save(model.state_dict(), 'model.pth')

model.load_state_dict(torch.load('model.pth'))

torch.save(model, 'model_complete.pth')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = model.to(device)

In [ ]:
# 梯度检查
for name, param in model.named_parameters():
    if param.requires_grad:
        print(f"{name}: {param.grad}")

# 冻结某些层
for param in model.fc.parameters():
    param.requires_grad = False

# 自定义层，注册到参数列表
class MyCustomLayer(nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(out_features, in_features))

    def forward(self, x):
        return x @ self.weight.t()

# 残差连接
class ResidualBlock(nn.Module):
    def __init__(self, channels):
        super().__init__()
        self.conv1 = nn.Conv2d(channels, channels, 3, padding=1)
        self.conv2 = nn.Conv2d(channels, channels, 3, padding=1)
    
    def forward(self, x):
        residual = x
        x = F.relu(self.conv1(x))
        x = self.conv2(x)
        return F.relu(x + residual)  # 残差连接

# 批量归一化和 dropout
class NetWithBN(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(784, 256)
        self.bn1 = nn.BatchNorm1d(256)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(256, 10)
    
    def forward(self, x):
        x = F.relu(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        return self.fc2(x)